[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/srchd/machine-learning/blob/visu/setting-up-visu/plane/visu.ipynb)

In [ ]:
import os
import pickle

import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dropout, Input, Conv2D, AveragePooling2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.optimizers

!wget -nc -P datasets/ http://nipg12.inf.elte.hu/~vavsaai@nipg.lab/annbsc21_p1_hw3/VOC2012/class_data.pkl
!wget -nc -P datasets/ http://nipg12.inf.elte.hu/~vavsaai@nipg.lab/annbsc21_p1_hw3/VOC2012/class_label.pkl

CAT_NAMES = ['aeroplane', 'bird', 'boat', 'bus', 'car', 'cat', 'cow', 'dog', 'horse', 'sheep']

with open('datasets/class_data.pkl', 'rb') as f:
    arr_cl_data = pickle.load(f)

with open('datasets/class_label.pkl', 'rb') as f:
    arr_cl_labs = pickle.load(f)

print(f'Shape of classification image array: {arr_cl_data.shape}')
print(f'Shape of classification label array: {arr_cl_labs.shape}')

In [ ]:
sample_idxs = np.random.choice(arr_cl_data.shape[0], size=12)

fig, axs = plt.subplots(2,6)
fig.set_figwidth(14)
fig.set_figheight(6)

for plot_idx in range(12):
    axs[plot_idx//6, plot_idx%6].imshow(arr_cl_data[sample_idxs[plot_idx]])
    text = CAT_NAMES[np.argmax(arr_cl_labs[sample_idxs[plot_idx]])]
    axs[plot_idx//6, plot_idx%6].axis("off")
    axs[plot_idx//6, plot_idx%6].set_title(str(text))